# Problem 1
## setting parameter
* There are $N$ product. $i \in \{0, 1,..., N-1\}$
* There are $M$ month. $j \in \{0, 1,..., M-1\}$, month 0, 1, 2... = March, April, March ...
* There are $K$ shipping method. $k \in \{0, 1,..., K-1\}$, shipping method 1, 2, 3 = express, air, Ocean
* Let $D_{ij}$ be the forecast demand of product $i$ of month $j$.
* Let $I_i$ be the quantity of initial inventory of product $i$.
* Let $BuyCost_i$ be the purchase cost of product $i$.
* Let $HoldCost_i$ be the holding cost of product $i$.
* Let $Transit_{ij}$ be the in-transt product $i$ which will be delivered on $j$ month.\[here $j \in\{1,2\}$\]
* Let $FixedShipCost_k$ be the fixed cost of shipping method $k$.
* Let $VarShipCost_{ik}$ be the variable cost of product $i$ of shipping method $k$.
* Let $LeadTime_k$ be the lead time of shipping method $k$.
* Let $M_{big}$ be $\sum_{j=0}^{M-1} \sum_{i=0}^{N-1} D_{ij}$

## setting decision variables
* let $x_{ijk}$ be the quantity of product $i$ that are order at month $j$ by shipping method $k$.
* let $z_{jk}$ be $1$ if at month $j$ we use shipping method $k$ else $0$

## ultility expression
* let $ExprY_{ij}$ be the expression Inventory level of product $i$ of month $j$.

$j=0:\ I_i - D_{i0} \quad \forall i$

$j=1:\ I_i - \sum_{z=0}^1 D_{iz} + x_{i00} + Transit_{i1} \quad \forall i$

$j=2:\ I_i - \sum_{z=0}^2 D_{iz} + \sum_{z=0}^1 x_{iz0} + \sum_{z=0}^0 x_{iz1} + \sum_{z=1}^2 transit_{iz} \quad \forall i$

$j \ge 3:\ I_i - \sum_{z=0}^j D_{iz} + \sum_{z=0}^{j-1} x_{iz0} + \sum_{z=0}^{j-2} x_{iz1} + \sum_{z=0}^{j-3} x_{iz2} + \sum_{z=1}^2 transit_{iz} \quad \forall i$
    

## Objective function
$$Min.\quad purchase\ Cost + Shipping\ Cost + Inventory\ Cost$$
Purchase Cost = $\sum_{i=0}^{N-1} \sum_{j=0}^{M-1} \sum_{k=0}^{K-1}( x_{ijk}\times BuyCost_i)$

Shipping Cost:<br>
* Fixed Cost = $\sum_{j=0}^{M-1} \sum_{k=0}^{K-1} (z_{jk} \times FixedShipCost_k)$

* Variable Cost = $\sum_{j=0}^{M-1} \sum_{i=0}^{N-1} \sum_{k=0}^{K-1} (x_{ijk} \times VarShipCost_{ik})$

Inventory Cost = $\sum_{i=0}^{N-1}[(\sum_{j=0}^{M-1} ExprY_{ij}) \times HoldCost_i]$

## Constrains

$ExprY_{ij} \ge 0 \quad \forall i, \forall j$

$\dfrac{\sum_{i=0}^{N-1}x_{ijk}}{M_{big}} \le z_{jk} \quad \forall j, \forall k$

$x_{ijk} \ge 0  \quad \forall i,\ \forall j,\ \forall k$

$z_{jk} \in \{0, 1\}$


In [1]:
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
import numpy as np

In [2]:
## setting parameter
N, M, K = 10, 6, 3
D_ij = pd.read_excel('data.xlsx', sheet_name='Demand', index_col='Product').to_numpy()
I_i = pd.read_excel('data.xlsx', sheet_name='Initial inventory', index_col='Product').to_numpy().squeeze()
BuyCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Purchasing cost'].to_numpy().squeeze()
HoldCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Holding'].to_numpy().squeeze()
Transit_ij = pd.read_excel('data.xlsx', sheet_name='In-transit', index_col='Product').to_numpy().squeeze()
FixedShipCost_k = (100, 80, 50)
VarShipCost_ik = pd.read_excel('data.xlsx', sheet_name='Shipping cost', index_col='Product').to_numpy().squeeze()
x_max = sum([sum(i) for i in D_ij])

In [3]:
m = gb.Model("Problem1Model")

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [4]:
#add decision variable
x = m.addVars(N, M, K, vtype=GRB.INTEGER, name='x_ijk')
z = m.addVars(M, K, vtype=GRB.BINARY, name='z_jk')

In [5]:
#set ExprY_ij
ExprY_ij = [
    [
        gb.LinExpr(I_i[i] - D_ij[i][0]),
        gb.LinExpr(I_i[i] - sum(D_ij[i][z] for z in range(0, 2)) + x[i,0,0] + Transit_ij[i][1]),
        gb.LinExpr(I_i[i] - sum(D_ij[i][z] for z in range(0, 3)) + sum(x[i,z,0] for z in range(0, 2)) + sum(x[i,z,1] for z in range(0, 1)) +  sum(Transit_ij[i][z] for z in range(1, 3)) )
     ] +
    [
        gb.LinExpr(
            I_i[i] - sum(D_ij[i][z] for z in  range(0, j+1)) +
            sum(x[i,z,0] for z in range(0, j)) +
            sum(x[i,z,1] for z in range(0, j-1)) +
            sum(x[i,z,2] for z in range(0, j-2)) +
            Transit_ij[i][1] + Transit_ij[i][2]
        )
    for j in range(3, M)
    ]
for i in range(N)]

In [6]:
#set objective function
purchaseCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for k in range(0, K))
        for j in range(0, M))
    for i in range(0, N))
)

FixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            z[j,k] * FixedShipCost_k[k]
        for k in range(0, K))
    for j in range(0, M))
)

VariableCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * VarShipCost_ik[i][k]
            for k in range(K))
        for i in range(N))
    for j in range(M))
)

InventoryCost = gb.LinExpr(
    quicksum(
        quicksum(
            ExprY_ij[i][j]
        for j in range(0, M)) * HoldCost_i[i]
    for i in range(0, N))
)

m.setObjective(
    purchaseCostExpr + FixedCost + VariableCost + InventoryCost
)

In [7]:
#Constrains
_ = m.addConstrs(
    ExprY_ij[i][j] >= 0
for i in range(0, N)
for j in range(0, M)
)

_ = m.addConstrs(
    quicksum(x[i,j,k] for i in range(0, N)) / x_max <= z[j,k]
for j in range(0, M)
for k in range(0, K)
)


In [8]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 78 rows, 198 columns and 508 nonzeros
Model fingerprint: 0x15531981
Variable types: 0 continuous, 198 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [5e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 7e+02]
Found heuristic solution: objective 1.700149e+07
Presolve removed 49 rows and 67 columns
Presolve time: 0.00s
Presolved: 29 rows, 131 columns, 301 nonzeros
Variable types: 0 continuous, 131 integer (11 binary)

Root relaxation: objective 1.649297e+07, 21 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6493e+07    0    4 1.7001e+07 1.6493e+07  2.99%     -    0s
H    0     0                    1.649315e+07 1.6493e+07  0

In [9]:
for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
print('Obj: %g' % m.objVal)

x_ijk[0,0,0] -0
x_ijk[0,0,1] -0
x_ijk[0,0,2] -0
x_ijk[0,1,0] -0
x_ijk[0,1,1] -0
x_ijk[0,1,2] -0
x_ijk[0,2,0] -0
x_ijk[0,2,1] -0
x_ijk[0,2,2] 38
x_ijk[0,3,0] -0
x_ijk[0,3,1] -0
x_ijk[0,3,2] -0
x_ijk[0,4,0] -0
x_ijk[0,4,1] -0
x_ijk[0,4,2] -0
x_ijk[0,5,0] -0
x_ijk[0,5,1] -0
x_ijk[0,5,2] -0
x_ijk[1,0,0] -0
x_ijk[1,0,1] -0
x_ijk[1,0,2] -0
x_ijk[1,1,0] -0
x_ijk[1,1,1] -0
x_ijk[1,1,2] -0
x_ijk[1,2,0] -0
x_ijk[1,2,1] -0
x_ijk[1,2,2] 45
x_ijk[1,3,0] -0
x_ijk[1,3,1] -0
x_ijk[1,3,2] -0
x_ijk[1,4,0] -0
x_ijk[1,4,1] -0
x_ijk[1,4,2] -0
x_ijk[1,5,0] -0
x_ijk[1,5,1] -0
x_ijk[1,5,2] -0
x_ijk[2,0,0] -0
x_ijk[2,0,1] -0
x_ijk[2,0,2] -0
x_ijk[2,1,0] -0
x_ijk[2,1,1] -0
x_ijk[2,1,2] 82
x_ijk[2,2,0] -0
x_ijk[2,2,1] -0
x_ijk[2,2,2] 200
x_ijk[2,3,0] -0
x_ijk[2,3,1] -0
x_ijk[2,3,2] -0
x_ijk[2,4,0] -0
x_ijk[2,4,1] -0
x_ijk[2,4,2] -0
x_ijk[2,5,0] -0
x_ijk[2,5,1] -0
x_ijk[2,5,2] -0
x_ijk[3,0,0] -0
x_ijk[3,0,1] -0
x_ijk[3,0,2] -0
x_ijk[3,1,0] -0
x_ijk[3,1,1] -0
x_ijk[3,1,2] 97
x_ijk[3,2,0] -0
x_ijk[3,2,1] -0
x_ijk[3